## **Lernziele**

Durch das Bearbeiten dieser Übungen werden Sie:

- Retrieval-Augmented Generation (RAG) und seine Komponenten verstehen.
- PDF-Dokumente effektiv laden, vorverarbeiten und verarbeiten.
- Textdaten in Embeddings für effiziente Suche umwandeln.
- Dokumenten-Retrieval-Systeme mit LangChain und FAISS implementieren und testen.
- Retrieval-Systeme mit kostenlosen Sprachmodellen (LLMs) von ChatGroq integrieren.
- Ein interaktives chatbasiertes Frage-Antwort-System aufbauen.

---

## **Übung 1: Setup und Aufwärmen**

In dieser Übung richten Sie Ihre Umgebung ein und wählen ein geeignetes Sprachmodell aus.

**Schritte:**

1. **Umgebungsvariablen laden:** Stellen Sie sicher, dass Ihre Umgebungsvariablen (z.B. API-Schlüssel, Tokens) sicher gespeichert und geladen werden.
2. **LLM auswählen:** Wählen Sie ein kostenloses LLM-Modell von ChatGroq aus.
3. **Modell instanziieren:** Erstellen Sie eine Instanz Ihres gewählten Modells.

In [12]:
import warnings
warnings.filterwarnings("ignore")

from dotenv import load_dotenv
load_dotenv()

True

In [13]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0.2,
    max_tokens=1024,
    timeout=60,
    max_retries=2
)

---

## **Übung 2: Datenaufnahme**

In dieser Übung lernen Sie, PDF-Daten in eine Python-Umgebung zu laden.

**Schritte:**

1. **PDF-Loader importieren:** Verwenden Sie LangChains `PyPDFLoader`.
2. **PDF-Datei laden:** Erstellen Sie eine Funktion zum Lesen der PDF-Datei.
3. **PDF-Inhalt anzeigen:** Geben Sie die Seitenanzahl und den Inhalt der ersten Seite aus.

In [14]:
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from pathlib import Path

data_path = Path("data")

loader = DirectoryLoader(
    str(data_path),
    glob="**/*.md",
    loader_cls=TextLoader,
    loader_kwargs={"encoding": "utf-8"},
)

docs = loader.load()

print(f"{len(docs)} Dokumente geladen")

23 Dokumente geladen


In [15]:
if len(docs) > 3:
    print(docs[3].page_content[:500])
    print("\n--- Metadaten ---")
    print(docs[3].metadata)
else:
    print(f"Nur {len(docs)} Dokument(e) vorhanden.")

# Tag 07, 27.10.2023

Logistische Regression

---

## Grundlegender Überblick

- **Überwachtes Lernen**: Nutzt gelabelte Daten zur Modellbildung.
- **Klassifikation vs. Regression**:
  - Klassifikation ordnet Eingaben diskreten Klassen zu (z.B. Spam/kein Spam).
  - Regression sagt kontinuierliche Werte voraus (z.B. Temperatur, Preis).
- **Klassifikationsarten**:
  - Binär: Zwei Klassen (z.B. traurig/glücklich).
  - Multiklassig: Mehr als zwei Klassen (z.B. Katze/Hund/Krokodil).
  - Multilabel: E

--- Metadaten ---
{'source': 'data\\protocol_26-01-27.md'}


---

## **Übung 3: Dokument-Chunking**

Diese Übung führt das Aufteilen großer Dokumente in handhabbare Textblöcke ein.

**Schritte:**

1. **Text-Splitter importieren:** Verwenden Sie `RecursiveCharacterTextSplitter`.
2. **Dokument aufteilen:** Schreiben Sie eine Funktion, die geladene Dokumente in Chunks aufteilt.
3. **Funktion testen:** Überprüfen Sie das Ergebnis, indem Sie die resultierenden Chunks anzeigen.

In [16]:
react_docs = docs

In [17]:
import re

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter, MarkdownHeaderTextSplitter
from langchain_core.documents import Document

def chunk_documents(documents, chunk_size=700, chunk_overlap=150):
    """
    Verbesserte Chunking-Funktion:
    - MarkdownHeaderTextSplitter extrahiert Überschriften als Metadaten
    - chunk_size=700 statt 200 fuer mehr Kontext pro Chunk
    - chunk_overlap=150 fuer bessere Kontinuitaet
    - Reichere Metadaten (Quelle, Zeichenanzahl, Header)
    """

    # FIX: Markdown-Überschriften als Metadaten nutzen
    headers_to_split_on = [
        ("#", "h1"),
        ("##", "h2"),
        ("###", "h3"),
    ]
    md_splitter = MarkdownHeaderTextSplitter(
        headers_to_split_on=headers_to_split_on,
        strip_headers=False
    )

    # FIX: Natürliche Trennzeichen priorisieren (kein harter Schnitt mitten im Satz)
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", ". ", " ", ""]
    )

    all_chunks = []
    for doc in documents:
        source_file = doc.metadata.get("source", "unknown")

        # Erst nach Markdown-Headings aufteilen
        md_chunks = md_splitter.split_text(doc.page_content)

        # Dann jeden MD-Chunk weiter aufteilen
        for md_chunk in md_chunks:
            sub_chunks = text_splitter.split_text(md_chunk.page_content)
            for sub_chunk in sub_chunks:
                days = re.findall(r"Tag \d{1,2}",sub_chunk)
                day_info=""
                if len(days)>0:
                    day_info=days[0]
                new_doc = Document(
                    page_content=sub_chunk,

                    metadata={
                        **doc.metadata,
                        **md_chunk.metadata,   # h1, h2, h3 aus Überschriften
                        "source_file": source_file,
                        "char_count": len(sub_chunk),
                        "day":day_info,
                        "day index":day_info,
                        "Was war an":day_info,
                        "day key":day_info,
                    }

                )
                all_chunks.append(new_doc)

    # FIX: IDs und Gesamtanzahl nach dem Erstellen aller Chunks setzen
    total = len(all_chunks)
    for i, chunk in enumerate(all_chunks):
        chunk.metadata.update({
            "id": f"chunk_{i}",
            "chunk_index": i,
            "total_chunks": total,
        })

    return all_chunks


react_chunks = chunk_documents(react_docs)

In [32]:
print(f"Anzahl Chunks: {len(react_chunks)}")
print("\n--- Beispiel-Chunk Inhalt ---")
print(react_chunks[0].page_content)
print("\n--- Beispiel-Chunk Metadaten ---")
print(react_chunks[0].metadata)

Anzahl Chunks: 470

--- Beispiel-Chunk Inhalt ---
# Tag 03, 21.01.2026  
Explorative Datenanalyse (kurz EDA)  
---

--- Beispiel-Chunk Metadaten ---
{'source': 'data\\protocol_26-01-21.md', 'h1': 'Tag 03, 21.01.2026', 'source_file': 'data\\protocol_26-01-21.md', 'char_count': 64, 'id': 'chunk_0', 'chunk_index': 0, 'total_chunks': 470}


In [ ]:
# # Import RecursiveCharacterTextSplitter
# from langchain_text_splitters import RecursiveCharacterTextSplitter

# def chunk_documents(documents, chunk_size=200, chunk_overlap=50):
#     text_splitter = RecursiveCharacterTextSplitter(
#         chunk_size=chunk_size,
#         chunk_overlap=chunk_overlap
#     )
#     chunks = text_splitter.split_documents(documents=documents)
    
#     # Just to add id for etch chunks to map it later 
#     for i, chunk in enumerate(chunks):
#          chunk.metadata.update({
#         "id": f"chunk_{i}",
#     })
    
#     return chunks


# react_chunks = chunk_documents(react_docs)

In [ ]:
# # Execute your chunking function and display results here
# len(react_chunks)


---

## **Übung 4: Embedding und Speicherung**

In dieser Übung erstellen Sie Embeddings aus Textblöcken und speichern sie effizient.

**Schritte:**

1. **Embedding-Modell wählen:** Verwenden Sie `sentence-transformers/all-mpnet-base-v2` von Hugging Face.
2. **Embeddings generieren:** Wandeln Sie Dokument-Chunks in Embeddings um.
3. **Embeddings speichern:** Speichern Sie diese Embeddings lokal mit FAISS.

In [33]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores.faiss import DistanceStrategy

# Embedding-Modell einmal global instanziieren
embedding_model = HuggingFaceEmbeddings(
    model_name='sentence-transformers/all-mpnet-base-v2',
    encode_kwargs={"normalize_embeddings": True}
)

def embed_and_store(chunks, db_name, embedding):
    vectorstore = FAISS.from_documents(
        documents=chunks,
        embedding=embedding,
        distance_strategy=DistanceStrategy.COSINE
    )
    vectorstore.save_local(f"../vector_databases/vector_db_{db_name}")
    print(f"Vektordatenbank gespeichert: ../vector_databases/vector_db_{db_name}")
    return vectorstore

In [34]:
# Globales Embedding-Modell übergeben
all_embedding = embed_and_store(chunks=react_chunks, db_name="chatbot", embedding=embedding_model)

Vektordatenbank gespeichert: ../vector_databases/vector_db_chatbot


---

## **Übung 5: Retrieval aus FAISS**

Hier lernen Sie, wie man Dokumente aus einer Vektordatenbank mithilfe von Embeddings abruft.

**Schritte:**

1. **Embeddings laden:** Laden Sie gespeicherte Embeddings aus der FAISS-Datenbank.
2. **Retrieval implementieren:** Erstellen Sie eine Logik zum Abrufen relevanter Chunks basierend auf Abfragen.
3. **Retriever testen:** Führen Sie die Suche mit Beispielabfragen durch.

In [ ]:
def retrieve_from_vector_db(vector_db_path, embedding):
    react_vectorstore = FAISS.load_local(
        folder_path=vector_db_path,
        embeddings=embedding,
        allow_dangerous_deserialization=True,
        distance_strategy=DistanceStrategy.COSINE
    )

    retriever = react_vectorstore.as_retriever(
        search_type="mmr",
        search_kwargs={"k": 6, "fetch_k": 20}
    )
    return retriever, react_vectorstore


react_retriever, react_vectorstore = retrieve_from_vector_db(
    "../vector_databases/vector_db_chatbot",
    embedding=embedding_model
)

print(type(react_retriever), type(react_vectorstore))

In [ ]:
# Test your retrieval system with queries
react_retriever.get_relevant_documents("Anekdote",k=3)

---

## **Übung 6: Retrieval mit LLM verbinden**

Nun verbinden Sie das Dokument-Retrieval mit dem Sprachmodell.

**Schritte:**

1. **Retrieval-Chain erstellen:** Verknüpfen Sie Ihr Retrieval-System mit Ihrem instanziierten LLM.
2. **Chain testen:** Bestätigen Sie die Funktionalität, indem Sie Antworten aus abgerufenen Dokumenten generieren.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

system_prompt = """Du bist ein präziser Assistent, der Fragen ausschließlich auf Basis der bereitgestellten Dokumente beantwortet.

Regeln:
- Beziehe dich konkret auf den bereitgestellten Kontext
- Wenn die Antwort nicht im Kontext enthalten ist, antworte exakt:
  "Diese Information ist in den bereitgestellten Dokumenten nicht enthalten."
- Nenne am Ende deiner Antwort die verwendete Quelldatei aus den Metadaten, sofern vorhanden

Kontext:
{context}
"""

prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),
])

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain

def connect_chains(retriever, prompt):
    stuff_documents_chain = create_stuff_documents_chain(
        llm=llm,
        prompt=prompt
    )
    retrieval_chain = create_retrieval_chain(
        retriever=retriever,
        combine_docs_chain=stuff_documents_chain
    )
    return retrieval_chain


react_retrieval_chain = connect_chains(react_retriever, prompt_template)

In [ ]:
# FIX: Hilfsfunktion für saubere Ausgabe – vermeidet Wiederholung im Code
def ask(question, chain=react_retrieval_chain):
    """Stellt eine Frage an die RAG-Chain und gibt die Antwort formatiert aus."""
    result = chain.invoke({"input": question})
    print(f"Frage: {question}")
    print(f"\nAntwort:\n{result['answer']}")
    print("\n" + "=" * 60 + "\n")
    return result

In [ ]:
ask("Was ist das Takeaway?")

In [ ]:
ask("Fasse Feature Engineering zusammen.")

---

## **Übung 7: Interaktives Chat-System**

In der letzten Übung bauen Sie ein interaktives chatbasiertes Abfragesystem.

**Schritte:**

1. **Chat-Oberfläche erstellen:** Entwickeln Sie eine einfache Funktion für interaktive Abfragen.
2. **Chat ausführen:** Ermöglichen Sie es Nutzern, Fragen zu stellen und sofortige Antworten zu erhalten.

In [ ]:
print("RAG-Chatbot gestartet. Tippe 'quit' oder 'exit' zum Beenden.\n")

while True:
    user_query = input("Deine Frage: ").strip()

    if not user_query:
        continue

    if user_query.lower() in ("quit", "exit", "q"):
        print("Chatbot beendet.")
        break

    try:
        result = react_retrieval_chain.invoke({"input": user_query})
        print(f"\nAntwort:\n{result['answer']}\n")
        print("-" * 60)
    except Exception as e:
        print(f"Fehler bei der Anfrage: {e}\nBitte versuche es erneut.\n")

In [ ]:
# Run and test your interactive chat system

---

## **Fazit & Reflexion**

Nach Abschluss dieser Übungen:

- Fassen Sie die gelernten Schlüsselkonzepte zusammen.
- Reflektieren Sie über die Wirksamkeit und Einschränkungen des kostenlosen LLM und des von Ihnen aufgebauten RAG-Systems.
- Überlegen Sie, wie Sie Ihr System in praktischen Anwendungen verbessern oder erweitern könnten.

---